In [24]:
import torch
import numpy as np
from torch_geometric.loader import DataLoader

from Models.GNN import IGCNet
from Utils.data_gen import Generate_Input, Generate_Label, proc_data
from Utils.training import sr_loss_matrix
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Parameters

In [14]:
num_H = 200
num_test = 20
K = 20 # number of terminals
M = 50 # number of access points
B = 20 # Mhz
D = 1 # km
tau=10
random_matrix = np.random.randn(tau, tau)
U, S, V = np.linalg.svd(random_matrix)

Hb = 15 # Base station height in m
Hm = 1.65 # Mobile height in m
f = 1900 # Frequency in MHz
aL = (1.1 * np.log10(f) - 0.7) * Hm - (1.56 * np.log10(f) - 0.8)
L = 46.3+33.9*np.log10(f)-13.82*np.log10(Hb)-aL

power_f=0.2 # downlink power
# Pd = power_f / 10 ** ((-203.975 + 10 * np.log10(20 * 10 ** 6) + 9) / 10) # normalized receive SNR
Ther_noise = 20000000 * 10**(-17.4) * 10**-3
Pd = 1/Ther_noise
Pu=Pd

d0=0.01 # km
d1=0.05 # km

N = 50

R_cf_min = np.zeros(N)  # Cell Free
R_cf_sum = np.zeros(N)
R_cf_opt_min = np.zeros(N)

In [15]:
# Generate Phii for num_H samples
Phii_All = np.zeros((num_H,K,tau))
for ite in range(num_H):
  Phii = np.zeros((K,tau))
  for k in range(K):
    Point = k % tau
    # Point = np.random.randint(1, tau+1)
    Phii[k,:] = U[Point - 1,:]
  Phii_All[ite,:,:] = Phii

Phii_Test = np.zeros((num_test,K,tau))
for ite in range(num_test):
  Phii = np.zeros((K,tau))
  for k in range(K):
    Point = np.random.randint(1, tau+1)
    Phii[k,:] = U[Point - 1,:]
  Phii_Test[ite,:,:] = Phii

BETAA_ALL = Generate_Input(num_H, tau, K, M, Pd, D=1, Hb=15, Hm=1.65, f=1900,
                    var_noise=1, Pmin=0, seed=2017, d0=d0, d1=d1)
BETAA_Test = Generate_Input(num_test, tau, K, M, Pd, D=1, Hb=15, Hm=1.65, f=1900,
                    var_noise=1, Pmin=0, seed=2017, d0=d0, d1=d1)
Eta_All = Generate_Label(num_H, tau, K, M, BETAA_ALL,Phii_All, Pu, Pd)
Eta_Test = Generate_Label(num_test, tau, K, M, BETAA_Test,Phii_Test, Pu, Pd)

In [16]:
train_data_list_all = proc_data(BETAA_ALL, Phii_All, Eta_All, Pd)
test_data_list_all = proc_data(BETAA_Test,Phii_Test,Eta_Test, Pd)
selectedAP = 0
train_data_list = []
test_data_list = []
for m in range(M):
  train_data_list.append([train_data_list_all[i+m] for i in range(0,num_H*M,M)]) # select the first AP to train the model
  test_data_list.append([test_data_list_all[i+m] for i in range(0,num_test*M,M)])
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(torch.cuda.is_available())

True


In [21]:
train_loader = [[] for _ in range(M)]
test_loader = [[] for _ in range(M)]
for m in range(M):
  train_loader[m] = list(DataLoader(train_data_list[m], batch_size=32, shuffle=False,num_workers=0))
  test_loader[m] = list(DataLoader(test_data_list[m], batch_size=32, shuffle=False,num_workers=0))
# for m in range(M):
#   train_loader[m] = list(train_loader[m])
#   test_loader[m] = list(test_loader[m])

In [ ]:
sr_weight = 0.2
nsu_weight = 0.8
Rthr = 3

# Load your model
# %cd '/content/drive/MyDrive/Mylearning/GNN_FL_based_Cell_Free_M_MIMO(Personal)'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = IGCNet().to(device)
# model.load_state_dict(torch.load('Decen_10Antens_all.pth'))

# model = IGCNet().to(device)
# use_gpu = torch.cuda.is_available()
# if use_gpu:
#     model = model.cuda()


optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.9)

LossSave=[]
TestLossSave=[]
for epoch in range(1,100):
  model.train()

  total_loss_train = 0
  for ite in range(len(train_loader[0])): # len(train_loader[0]) ~ number of batch
    data_trainn = []
    out_trainn = []
    for m in range(M): # 2
      data = train_loader[m][ite]
      data = data.to(device)
      data_trainn.append(data)
      if m==0:
        out_trainn.append(model(data))
      else:
        with torch.no_grad():
            out_trainn.append(model(data))

    # Loss calculation
    loss,_,_ = sr_loss_matrix(data_trainn, out_trainn, K, M, sr_weight, nsu_weight, Rthr, tau)
    loss.backward()
    total_loss_train += loss.item() * train_loader[m][ite].num_graphs
    optimizer.step()
    optimizer.zero_grad()
  loss1 = total_loss_train/num_H*-1
  LossSave.append(loss1)
  if (epoch % 1 ==0):
    model.eval()

    total_loss_test = 0
    sumrate_test = 0
    num_satisfied_test = 0
    for ite in range(len(test_loader[0])):
      data_testt = []
      out_testt = []
      for m in range(M):
        data_testt.append(test_loader[m][ite])
        out_testt.append(model(test_loader[m][ite].to(device)))
      # Loss calculation
      with torch.no_grad():
        loss,sumrate,num_satisfied = sr_loss_matrix(data_testt, out_testt, K, M, sr_weight, nsu_weight, Rthr)
        # print(f"loss test: {loss.item()}")
        total_loss_test += loss.item() * test_loader[m][ite].num_graphs
        sumrate_test += sumrate.item() * test_loader[m][ite].num_graphs
        num_satisfied_test += num_satisfied.item() * test_loader[m][ite].num_graphs
        # print(f"total_loss_test: {total_loss_test}")
    # print(num_test)
    loss2 = total_loss_test/num_test*-1
    sumrate2 = sumrate_test/num_test
    num_satisfied2 = num_satisfied_test/num_test
    TestLossSave.append(loss2)
    print('Epoch {:03d}, Train Loss: {:.4f}, Val Loss: {:.4f}, Sum rate: {:.4f}, Number of satisfied Users: {:.4f}'.format(
            epoch, loss1, loss2, sumrate2, num_satisfied2))
    scheduler.step()

/home/wine-4/miniconda3/envs/GNN_FL/lib/python3.10/site-packages/torch_geometric/warnings.py:17: UserWarning: The usage of `scatter(reduce='max')` can be accelerated via the 'torch-scatter' package, but it was not found
  warnings.warn(message)


NameError: name 'tau' is not defined